In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd


from sklearn.ensemble  import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression as logreg

import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sma

from sklearn.feature_selection import chi2

import scipy as sp
from scipy import stats

from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
def filterPerfectCorrelation(df):
    cols = df.columns
    removeCols = []
    for i in range(len(cols)-1):
        for j in range(i+1,len(cols)):
            cor = stats.pearsonr(df[cols[i]].values, df[cols[j]].values)[0]
            if cor >=1 or cor <=-1:
                removeCols.append(cols[i])
    return df[[c for c in cols if c not in removeCols]]
        

In [4]:
def bolleanToNumeric(df):
    for c in df.columns:
        df[c] = df[c].astype(int)
    return df

In [5]:
# remove class imballance by sampling from majority class
def classImballanceDownSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    minClassSize = np.min(counts)
    for clas in classes:
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(n=minClassSize, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df  

In [6]:
# remove class imballance by up sampling minority classes
def classImballanceUpSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    maxClassSize = np.max(counts).astype(float)
    fractions = maxClassSize / counts
    for clas,f in zip(classes,fractions):
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(frac=f, replace=True, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df  

In [7]:
# model that trains a default random forest and prints the report
def getModel(df,label, ballanceClasses=None, rfcBalance = None):
    
    xcols = [c for c in df.columns if c != label]
    ycols = [label]
    print("Predictors: ",len(xcols))
    print("label: ",ycols[0])
    
#     if ballanceClasses != None:
#         df = ballanceClasses(df, ycols[0])
        
    X = df[xcols]
    Y = df[ycols]
    
    #X = labelEncodeCategoricalFeatures(X)
    #print("Label Encoding.")
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=True)
    print("Train Test Split:")
    print(np.array(X_train).shape, np.array(X_test).shape, np.array(y_train).shape, np.array(y_test).shape)
    
    if ballanceClasses != None:
        dfTrain = X_train
        dfTrain['label'] = y_train
        dfTrain = ballanceClasses(dfTrain,'label')
        X_train = dfTrain[ [c for c in dfTrain.columns if c not in 'label'] ]
        y_train = dfTrain['label']
    
    #print("Checking if sampling worked:")
    #print(y_train.value_counts())
    #print()
    
    if rfcBalance == None:
        clf = rfc()
    else:
        clf = rfc(class_weight = rfcBalance)
        print(clf)
    
    #clf = rfc()
    clf.fit(X_train, y_train.values.ravel())
    y_pred = clf.predict(X_test)
    print("Accuracy : ", np.sum(y_test[ycols[0]].values==y_pred).astype(float)/len(y_test) )
    print(classification_report(y_test[ycols[0]].values, y_pred))

In [8]:
# baseline model
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(28, input_dim=28, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

### Load Data

In [9]:
cleandf = pd.read_csv('cleanData.csv')

In [10]:
cleandf.head()

,ads18_frequentHealthIssue,ads18_highBP,ads18_highChl,ads18_unknownChl,ads18_asthma,ads18_formerasthma,ads18_white,ads18_weight,ads18_obese,ads18_havechild,...,ads18_male,ads18_healthCov,ads18_older40,ads18_age18_25,ads18_age25_30,ads18_seatbelt,ads18_veteran,ads18_affordDoc,ads18_stroke,label
0,False,True,False,False,False,False,True,8890.0,True,False,...,True,True,True,False,False,True,True,True,False,False
1,False,False,False,True,False,False,True,7484.0,True,False,...,True,True,True,False,False,True,True,True,False,False
2,False,False,True,False,False,False,True,5897.0,False,False,...,False,True,True,False,False,False,False,True,False,False
3,False,False,False,False,False,False,True,11340.0,True,True,...,True,True,False,False,True,False,False,True,False,False
4,False,False,False,True,False,False,True,4536.0,False,True,...,False,False,False,False,True,False,False,False,False,False


In [11]:
print("Records: ",len(cleandf))
cleandf = cleandf.dropna()
print("Records after cleaning Nan: ",len(cleandf))

Records:  61934
Records after cleaning Nan:  55953


In [12]:
cleandf = classImballanceDownSample(cleandf, 'label')

Before Class Imballance Treatment: 
False    50092
True      5861
Name: label, dtype: int64
After Class Imballance Treatment: 
True     5861
False    5861
Name: label, dtype: int64


In [13]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [14]:
Y= cleandf['label']
X = np.asarray(cleandf[[c for c in cleandf.columns if c != 'label']])
#encoded_Y = Y

In [15]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [16]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=20, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Results: 55.85% (7.15%)
